# Parte 3: Recolección de información
### Web Scraping a "Semana" para investigación referente a los cambios climáticos en Colombia

### Paso 1: Importación de librerias

In [1]:
import pandas as pd
import time
import random
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support import expected_conditions as EC
from random import shuffle

### Paso 2: Definición de opciones para el navegador

In [2]:
chrome_options = Options()
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument("--start-maximized")
chrome_options.add_argument('--window-size=1420,1080')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument('--disable-gpu')
chrome_options.add_argument("--disable-notifications")
chrome_options.add_argument("--remote-debugging-port=9222")
chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
chrome_options.add_experimental_option('useAutomationExtension', False)
chrome_options.add_experimental_option("excludeSwitches", ["disable-popup-blocking"])

### Paso 3: Definición de clase personalizada

In [3]:
# Clase para personalización
class bcolors:
    WARNING = '\033[93m'
    FAIL = '\033[91m'

# Control de errores
try:   
    # Definición de función
    def Seconds(result):
        hours = round(result/3600)
        minutes = round((result - (hours * 3600))/60)
        seconds = round(result - ((hours * 3600) + (minutes * 60)))
        if seconds < 0: seconds = seconds*-1
        if minutes < 0: minutes = minutes*-1
        final_time = "El sitema tardó: " + str(hours) + " horas " + str(minutes) + " minutos " + str(seconds) + " segundos en finalizar su ejeción"
        return final_time
    
except Exception as e: 
    print(bcolors.FAIL + "Revisa el error => ", e) # Impresión del error personalizado

### Paso 4: Lectura de archivo de noticias

In [4]:
# Definición de la clase
class IterableClass:
    
    # Función inicial o constructor
    def __init__(self, list_values = None): # None es para que sea opcional
        self.list_values = list_values
        self.dateframe = pd.read_csv(r'df_clean_7.csv') # lectura del archivo
        self.i = 0
        
    # Retornar un iterador
    def __iter__(self):
        return self
    
    # Retorna un elemento del contenedor
    def __next__(self):
        if self.i >= len(self.list_values):
            print("\nNo hay más valores a recorrer... ⬇")
            raise StopIteration
            
        item = self.list_values[self.i]
        self.i += 1
        return item  

### Paso 5: Instacia del objeto e impresión del DataFrame

In [5]:
# Instancia del objeto
IC = IterableClass()

# 5.1. Eliminación de columna que se creo en por error "Unnamed: 0" .drop('Unnamed: 0', axis=1)
# 5.2. Reseteo de index por la acción de combinación de DataFrames .reset_index().drop(['index'], axis=1)
IC.dateframe.drop('Unnamed: 0', axis=1).reset_index().drop(['index'], axis=1)

,headers_text,headers_link
0,Choques entre campesinos y el Esmad por proyec...,https://www.semana.com/esmad-y-campesinos-choc...
1,"¿Qué pasa con los Cerros Orientales?\r\nmar, 8...",https://www.semana.com/cerros-orientales-que-o...
2,Calcular nuestro impacto en la naturaleza y có...,https://www.semana.com/pacto-biodiversidad-y-h...
3,Avanzan labores para salvar a jaguares y pumas...,https://www.semana.com/corpoamazonia-busca-ayu...
4,"“El petróleo amenaza el páramo, el agua y la s...",https://www.semana.com/alcalde-de-pasca-petrol...
...,...,...
239,Cinco nuevos saltamontes descubiertos en las s...,https://www.semana.com/medio-ambiente/articulo...
240,Descuartizan oso para amenazar a Parques Nacio...,https://www.semana.com/oso-de-anteojos-descuar...
241,El río que no se deja llevar por la corriente\...,https://www.semana.com/el-rio-que-no-se-deja-l...
242,¿Qué hacer con la gente que vive en los páramo...,https://www.semana.com/paramos-que-hacer-con-l...


### Paso 6: Definición de función para unificar los diferentes parrafos de la noticia

In [6]:
def JoinParagraphs(paragraphs):
    TextParagraphs = ""
    for paragraph in paragraphs:
        TextParagraphs = TextParagraphs + paragraph.text
        
    return TextParagraphs

### Paso 7: Definición de función para unificar los diferentes temas de la noticia

In [7]:
def JoinThemes(themes):
    TextThemes = ""
    for theme in themes:
        TextThemes = TextThemes + theme.text + ", "
    return TextThemes

### Paso 8: Recolección de información para cada noticia

In [8]:
# 8.1 Declaraciones de variables
titles = []
summaries = []
authors = []
dates = []
paragraphs = []
themes = []

In [9]:
# PASO 8.2: Aviso para el usuario final de que se inicio el web-scraping --------------------------------------------------------------#
print("Se inició el Web Scraping a 'Semana', por favor espere...") # Mensaje para visualización del inicio del proceso

# 8.2.1 Variable para definir el inicio de tiempo en ejecución
start = time.time()

Se inició el Web Scraping a 'Semana', por favor espere...


In [10]:
# PASO 8.3 Estructura base para acceder a la página en cuestión ----------------------------------------------------------------------#
service = Service(executable_path=r'../../chromedriver-win64/chromedriver.exe') # Creación del servicio mediante el driver
options = webdriver.ChromeOptions() # Definición de variable con las opciones para chrome
driver = webdriver.Chrome(service=service, options=options) # Creación del objeto del WebDriver

# Asignación de dataframe a una variable
links = IC.dateframe

# Control del error al momento de finalizar el páginado
try:
    
    # Iteración para acceso a los links del dataframe
    for i in range(len(links)):
        
        news = {
            "titles" : [],
            "summaries" : [],
            "authors" : [],
            "dates" : [],
            "paragraphs" : [],
            "themes" : [],
        }
        
        # Acceso a cada link del dataframe
        driver.get(links.iloc[i]['headers_link'])

        # Títulos
        # ****************************************************************************************************************
        title = driver.find_elements(By.XPATH, '//*[@id="fusion-app"]/div/main/article/div/div[2]/h1')
        if title:
            for i in title:
                print(titles.append(i.text))
        else: titles.append('...')
        # ****************************************************************************************************************

        # Resúmenes
        # ****************************************************************************************************************
        summary = driver.find_elements(By.XPATH, '//*[@id="fusion-app"]/div/main/article/div/div[2]/p')
        if summary: 
            for i in summary:
                print(summaries.append(i.text))
        else: summaries.append('...')
        # ****************************************************************************************************************

        # Autor
        # ****************************************************************************************************************
        date_a = driver.find_elements(By.XPATH, '//*[@id="fusion-app"]/div/main/article/div/div[2]/div[2]/div')
        if date_a: 
            if len(date_a) == 1:
                author_a = driver.find_elements(By.XPATH, '//*[@id="fusion-app"]/div/main/article/div/div[2]/div[2]/a/div')
                if author_a:
                    for i in author_a:
                        print(authors.append(i.text))
                else: authors.append('...')
            elif len(date_a) > 1:
                author_b = driver.find_elements(By.XPATH, '//*[@id="fusion-app"]/div/main/article/div/div[2]/div[2]/div[1]')
                if author_b:
                    for i in author_b:
                        print(authors.append(i.text))
                else: authors.append('...')
            else: authors.append('...')
        else: authors.append('...')
        # Test
        author_a = None
        author_b = None
        date_a = None
        # ****************************************************************************************************************

        # Fecha
        # ****************************************************************************************************************
        date_a = driver.find_elements(By.XPATH, '//*[@id="fusion-app"]/div/main/article/div/div[2]/div[2]/div')
        if date_a: 
            if len(date_a) == 1:
                if date_a:
                    for i in date_a:
                        print(dates.append(i.text))
                else: dates.append('...')
            elif len(date_a) > 1:
                date_b = driver.find_elements(By.XPATH, '//*[@id="fusion-app"]/div/main/article/div/div[2]/div[2]/div[2]')
                if date_b:
                    for i in date_b:
                        print(dates.append(i.text))
                else: dates.append('...')
            else: dates.append('...')
        else: dates.append('...')
        # Test
        author_a = None
        date_a = None
        date_b = None
        # ****************************************************************************************************************
        
        # Parrafos
        # ****************************************************************************************************************
        paragraphs_a = driver.find_elements(By.CLASS_NAME, 'prose')
        if paragraphs_a: 
            if len(paragraphs_a) > 1:
                paragraphs.append(JoinParagraphs(paragraphs_a))
            else:
                paragraphs.append(paragraphs_a[0].text)
        else: paragraphs.append('...')
        # ****************************************************************************************************************

        # Temas
        # ****************************************************************************************************************
        tags = driver.find_elements(By.CSS_SELECTOR, 'a.inline-block')
        if tags: 
            if len(tags) > 1:
                themes.append(JoinThemes(tags))
            else:
                themes.append(tags[0].text)
        else: themes.append('...')
        # ****************************************************************************************************************
                
        # Construcción del diccionario 
        news = {
            "titles" : titles,
            "summaries" : summaries,
            "authors" : authors,
            "dates" : dates,
            "paragraphs" : paragraphs,
            "themes" : themes
        }

        # Construcción inicial del dataframe
        df = pd.DataFrame(news)

        # (Opcional) Modo DEBUG, sirve para validar como el DataFrame se está llenando según las iteraciones
        display(df)
    else:
        driver.quit();
        
except Exception as e:
    display("Exception :", e)
    driver.quit() 
except NoSuchElementException as nse:
    display("NoSuchElementException :", nse)
    driver.quit()
except TimeoutException as tim:
    display("TimeoutException :", tim)
    driver.quit()

None
None
None


,titles,summaries,authors,dates,paragraphs,themes
0,Choques entre campesinos y el Esmad por proyec...,Agentes de la Fuerza Pública desalojaron a var...,...,24 de julio de 2017,En la mañana de este lunes se escribió un nuev...,"Esmad, Explotación petrolera, Exploración petr..."


None
None
None


,titles,summaries,authors,dates,paragraphs,themes
0,Choques entre campesinos y el Esmad por proyec...,Agentes de la Fuerza Pública desalojaron a var...,...,24 de julio de 2017,En la mañana de este lunes se escribió un nuev...,"Esmad, Explotación petrolera, Exploración petr..."
1,¿Qué pasa con los Cerros Orientales?,"Le explicamos en tres pasos, y en menos de un ...",...,8 de agosto de 2017,LOS CERROS ORIENTALES Y LA FRANJA DE ADECUACIÓ...,"Cerros orientales, Secretaría de Ambiente de B..."


None
None
None


...

,titles,summaries,authors,dates,paragraphs,themes
0,Choques entre campesinos y el Esmad por proyec...,Agentes de la Fuerza Pública desalojaron a var...,...,24 de julio de 2017,En la mañana de este lunes se escribió un nuev...,"Esmad, Explotación petrolera, Exploración petr..."
1,¿Qué pasa con los Cerros Orientales?,"Le explicamos en tres pasos, y en menos de un ...",...,8 de agosto de 2017,LOS CERROS ORIENTALES Y LA FRANJA DE ADECUACIÓ...,"Cerros orientales, Secretaría de Ambiente de B..."
2,Calcular nuestro impacto en la naturaleza y có...,Pacto Biodiversidad y Huella Forestal son dos ...,...,18 de julio de 2017,La presidente y fundadora de la asociación fra...,"Pacto Biodiversidad, Huella de carbono, Consum..."
3,Avanzan labores para salvar a jaguares y pumas...,En los últimos días se han reportado numerosos...,...,18 de julio de 2017,"La deforestación, los cultivos ilícitos y la g...","Jaguar, Puma, Felinos, Campesinos, Corpoamazon..."
4,"“El petróleo amenaza el páramo, el agua y la s...","El alcalde de Pasca, Carlos Muñoz, explica por...",...,24 de mayo de 2017,El auge de las consultas populares sigue creci...,"Consultas populares, Explotación petrolera, Pe..."
...,...,...,...,...,...,...
237,“Programas REDD+ deben incluir a quienes histó...,Chris Van Dam de la ONG Forest Trends asegura ...,...,23 de octubre de 2016,Bosques tropicales\nConservación\nWWF\nPlaneta...,"Bosques tropicales, Conservación, WWF, Planeta..."
238,‘Aviturismo’ en Perú: mirarlas en vez de matarlas,Perú al igual que Colombia se está convirtiend...,...,16 de noviembre de 2016,Aves\nObservadores de aves\nAviturismo\nPajare...,"Aves, Observadores de aves, Aviturismo, Pajare..."
239,Cinco nuevos saltamontes descubiertos en las s...,El hecho se presentó en la Reserva Natural Río...,...,7 de octubre de 2016,Biodiversidad\nWWF\nPlaneta vivo\nNariño\nArtr...,"Biodiversidad, WWF, Planeta vivo, Nariño, Artr..."
240,Descuartizan oso para amenazar a Parques Nacio...,A la sede de Parques Nacionales Naturales en e...,...,26 de septiembre de 2016,Parques Nacionales Naturales\nOso de anteojos\...,"Parques Nacionales Naturales, Oso de anteojos,..."


None
None
None


,titles,summaries,authors,dates,paragraphs,themes
0,Choques entre campesinos y el Esmad por proyec...,Agentes de la Fuerza Pública desalojaron a var...,...,24 de julio de 2017,En la mañana de este lunes se escribió un nuev...,"Esmad, Explotación petrolera, Exploración petr..."
1,¿Qué pasa con los Cerros Orientales?,"Le explicamos en tres pasos, y en menos de un ...",...,8 de agosto de 2017,LOS CERROS ORIENTALES Y LA FRANJA DE ADECUACIÓ...,"Cerros orientales, Secretaría de Ambiente de B..."
2,Calcular nuestro impacto en la naturaleza y có...,Pacto Biodiversidad y Huella Forestal son dos ...,...,18 de julio de 2017,La presidente y fundadora de la asociación fra...,"Pacto Biodiversidad, Huella de carbono, Consum..."
3,Avanzan labores para salvar a jaguares y pumas...,En los últimos días se han reportado numerosos...,...,18 de julio de 2017,"La deforestación, los cultivos ilícitos y la g...","Jaguar, Puma, Felinos, Campesinos, Corpoamazon..."
4,"“El petróleo amenaza el páramo, el agua y la s...","El alcalde de Pasca, Carlos Muñoz, explica por...",...,24 de mayo de 2017,El auge de las consultas populares sigue creci...,"Consultas populares, Explotación petrolera, Pe..."
...,...,...,...,...,...,...
238,‘Aviturismo’ en Perú: mirarlas en vez de matarlas,Perú al igual que Colombia se está convirtiend...,...,16 de noviembre de 2016,Aves\nObservadores de aves\nAviturismo\nPajare...,"Aves, Observadores de aves, Aviturismo, Pajare..."
239,Cinco nuevos saltamontes descubiertos en las s...,El hecho se presentó en la Reserva Natural Río...,...,7 de octubre de 2016,Biodiversidad\nWWF\nPlaneta vivo\nNariño\nArtr...,"Biodiversidad, WWF, Planeta vivo, Nariño, Artr..."
240,Descuartizan oso para amenazar a Parques Nacio...,A la sede de Parques Nacionales Naturales en e...,...,26 de septiembre de 2016,Parques Nacionales Naturales\nOso de anteojos\...,"Parques Nacionales Naturales, Oso de anteojos,..."
241,El río que no se deja llevar por la corriente,Existe un plan para construir 20 megahidroeléc...,Jack Lo Lau*,20 de septiembre de 2016,América Latina\nPerú\nUnirme al canal de Whats...,"América Latina, Perú,"


None
None
None


,titles,summaries,authors,dates,paragraphs,themes
0,Choques entre campesinos y el Esmad por proyec...,Agentes de la Fuerza Pública desalojaron a var...,...,24 de julio de 2017,En la mañana de este lunes se escribió un nuev...,"Esmad, Explotación petrolera, Exploración petr..."
1,¿Qué pasa con los Cerros Orientales?,"Le explicamos en tres pasos, y en menos de un ...",...,8 de agosto de 2017,LOS CERROS ORIENTALES Y LA FRANJA DE ADECUACIÓ...,"Cerros orientales, Secretaría de Ambiente de B..."
2,Calcular nuestro impacto en la naturaleza y có...,Pacto Biodiversidad y Huella Forestal son dos ...,...,18 de julio de 2017,La presidente y fundadora de la asociación fra...,"Pacto Biodiversidad, Huella de carbono, Consum..."
3,Avanzan labores para salvar a jaguares y pumas...,En los últimos días se han reportado numerosos...,...,18 de julio de 2017,"La deforestación, los cultivos ilícitos y la g...","Jaguar, Puma, Felinos, Campesinos, Corpoamazon..."
4,"“El petróleo amenaza el páramo, el agua y la s...","El alcalde de Pasca, Carlos Muñoz, explica por...",...,24 de mayo de 2017,El auge de las consultas populares sigue creci...,"Consultas populares, Explotación petrolera, Pe..."
...,...,...,...,...,...,...
239,Cinco nuevos saltamontes descubiertos en las s...,El hecho se presentó en la Reserva Natural Río...,...,7 de octubre de 2016,Biodiversidad\nWWF\nPlaneta vivo\nNariño\nArtr...,"Biodiversidad, WWF, Planeta vivo, Nariño, Artr..."
240,Descuartizan oso para amenazar a Parques Nacio...,A la sede de Parques Nacionales Naturales en e...,...,26 de septiembre de 2016,Parques Nacionales Naturales\nOso de anteojos\...,"Parques Nacionales Naturales, Oso de anteojos,..."
241,El río que no se deja llevar por la corriente,Existe un plan para construir 20 megahidroeléc...,Jack Lo Lau*,20 de septiembre de 2016,América Latina\nPerú\nUnirme al canal de Whats...,"América Latina, Perú,"
242,¿Qué hacer con la gente que vive en los páramos?,La Corte Constitucional prohibió tajantemente ...,...,6 de septiembre de 2016,Páramos\nMinisterio de Ambiente\nLuis Gilberto...,"Páramos, Ministerio de Ambiente, Luis Gilberto..."


In [11]:
df_news_clean = df.drop_duplicates(subset=["titles"])

In [12]:
df_news_clean.to_csv("df_news/df_news_clean_7.csv", escapechar='\\')

In [13]:
# Variable para definir el final de tiempo en ejecución
end = time.time()

# Variable para calcular tiempo y para enviar a la función
result = end-start

#Impresión del tiempo que tardó
print(Seconds(result))

El sitema tardó: 0 horas 15 minutos 1 segundos en finalizar su ejeción


In [14]:
df_news_clean

,titles,summaries,authors,dates,paragraphs,themes
0,Choques entre campesinos y el Esmad por proyec...,Agentes de la Fuerza Pública desalojaron a var...,...,24 de julio de 2017,En la mañana de este lunes se escribió un nuev...,"Esmad, Explotación petrolera, Exploración petr..."
1,¿Qué pasa con los Cerros Orientales?,"Le explicamos en tres pasos, y en menos de un ...",...,8 de agosto de 2017,LOS CERROS ORIENTALES Y LA FRANJA DE ADECUACIÓ...,"Cerros orientales, Secretaría de Ambiente de B..."
2,Calcular nuestro impacto en la naturaleza y có...,Pacto Biodiversidad y Huella Forestal son dos ...,...,18 de julio de 2017,La presidente y fundadora de la asociación fra...,"Pacto Biodiversidad, Huella de carbono, Consum..."
3,Avanzan labores para salvar a jaguares y pumas...,En los últimos días se han reportado numerosos...,...,18 de julio de 2017,"La deforestación, los cultivos ilícitos y la g...","Jaguar, Puma, Felinos, Campesinos, Corpoamazon..."
4,"“El petróleo amenaza el páramo, el agua y la s...","El alcalde de Pasca, Carlos Muñoz, explica por...",...,24 de mayo de 2017,El auge de las consultas populares sigue creci...,"Consultas populares, Explotación petrolera, Pe..."
...,...,...,...,...,...,...
239,Cinco nuevos saltamontes descubiertos en las s...,El hecho se presentó en la Reserva Natural Río...,...,7 de octubre de 2016,Biodiversidad\nWWF\nPlaneta vivo\nNariño\nArtr...,"Biodiversidad, WWF, Planeta vivo, Nariño, Artr..."
240,Descuartizan oso para amenazar a Parques Nacio...,A la sede de Parques Nacionales Naturales en e...,...,26 de septiembre de 2016,Parques Nacionales Naturales\nOso de anteojos\...,"Parques Nacionales Naturales, Oso de anteojos,..."
241,El río que no se deja llevar por la corriente,Existe un plan para construir 20 megahidroeléc...,Jack Lo Lau*,20 de septiembre de 2016,América Latina\nPerú\nUnirme al canal de Whats...,"América Latina, Perú,"
242,¿Qué hacer con la gente que vive en los páramos?,La Corte Constitucional prohibió tajantemente ...,...,6 de septiembre de 2016,Páramos\nMinisterio de Ambiente\nLuis Gilberto...,"Páramos, Ministerio de Ambiente, Luis Gilberto..."
